<a href="https://colab.research.google.com/github/pierre-brendan/PelotonIQ/blob/master/models/paris_nice_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First attempt at a model to predict the winner of Paris - Nice

In [0]:
# Mount drive to get cyclist data
#from google.colab import drive
#drive.mount('/content/drive')

In [139]:
# All this code is from the Kmeans clustering code

# load modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import preprocessing

# load the rider data
cycling_data = pd.read_csv('/content/drive/My Drive/cycling_data/rider_data.csv')
cycling_data = cycling_data[cycling_data['Rider'] != 'Bjorg Lambrecht ']
cycling_data = cycling_data[cycling_data['Rider'] != 'Bjorg Lambrecht']
cycling_data = cycling_data[cycling_data['Rider'] != 'Robbert de Greef ']
cycling_data = cycling_data[cycling_data['Rider'] != 'Robbert de Greef']
cycling_data = cycling_data[cycling_data['Rider'] != 'Could not find rider']

# Load the GC data
gc_winners = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/gc_winners_consolidated.csv')
gc_winners.rename(columns = {'Cyclist':'Rider'}, inplace = True)

# Merge the GC winners to the rider data set
cycling_data = pd.merge(cycling_data, gc_winners, how='left', on='Rider')

# Load in the Monuments data
san_remo = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/san_remo.csv')
san_remo = san_remo['san_remo_win'].groupby(san_remo['Rider']).sum()
roubaix = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/roubaix.csv')
roubaix = roubaix['roubaix_win'].groupby(roubaix['Rider']).sum()
flanders = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/flanders.csv')
flanders = flanders['flanders_win'].groupby(flanders['Rider']).sum()
lombardia = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/lombardia.csv')
lombardia = lombardia['lombardia_win'].groupby(lombardia['Rider']).sum()
liege = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/liege.csv')
liege = liege['lbl_win'].groupby(liege['Rider']).sum()
strade = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/strade.csv')
strade['strade_win'] = 1
strade = strade['strade_win'].groupby(strade['Rider']).sum()
suisse = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/suisse.csv')
suisse['suisse_win'] = 1
suisse = suisse['suisse_win'].groupby(suisse['Rider']).sum()
romandie = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/romandie.csv')
romandie['romandie_win'] = 1
romandie = romandie['romandie_win'].groupby(romandie['Rider']).sum()
paris_nice = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/paris_nice.csv')
paris_nice['paris_nice_win'] = 1
paris_nice = paris_nice['paris_nice_win'].groupby(paris_nice['Rider']).sum()
gent = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/gent.csv')
gent['gent_win'] = 1
gent = gent['gent_win'].groupby(gent['Rider']).sum()
fleche = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/fleche.csv')
fleche['fleche_win'] = 1
fleche = fleche['fleche_win'].groupby(fleche['Rider']).sum()
e3_binckbank = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/e3_binckbank.csv')
e3_binckbank['e3_binckbank_win'] = 1
e3_binckbank = e3_binckbank['e3_binckbank_win'].groupby(e3_binckbank['Rider']).sum()
dauphine = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/dauphine.csv')
dauphine['dauphine_win'] = 1
dauphine = dauphine['dauphine_win'].groupby(dauphine['Rider']).sum()
amstel = pd.read_csv('/content/drive/My Drive/cycling_data/Historical_GC_Classics_Results/amstel.csv')
amstel['amstel_win'] = 1
amstel = amstel['amstel_win'].groupby(amstel['Rider']).sum()


# Merge the Monuments winners to the rider data set
cycling_data = pd.merge(cycling_data, san_remo, how='left', on='Rider')
cycling_data = pd.merge(cycling_data, flanders, how='left', on='Rider')
cycling_data = pd.merge(cycling_data, liege, how='left', on='Rider')
cycling_data = pd.merge(cycling_data, lombardia, how='left', on='Rider')
cycling_data = pd.merge(cycling_data, roubaix, how='left', on='Rider')
cycling_data = pd.merge(cycling_data, strade, how='left', on='Rider')
cycling_data = pd.merge(cycling_data, suisse, how='left', on='Rider')
cycling_data = pd.merge(cycling_data, fleche, how='left', on='Rider')
cycling_data = pd.merge(cycling_data, gent, how='left', on='Rider')
cycling_data = pd.merge(cycling_data, paris_nice, how='left', on='Rider')
cycling_data = pd.merge(cycling_data, e3_binckbank, how='left', on='Rider')
cycling_data = pd.merge(cycling_data, amstel, how='left', on='Rider')
cycling_data = pd.merge(cycling_data, dauphine, how='left', on='Rider')
cycling_data = pd.merge(cycling_data, romandie, how='left', on='Rider')

# Replace NaN with 0's
cycling_data.fillna(0, inplace=True)

# Let's weed out the lower tiered teams for now
cycling_data['tmp_count'] = 1
ccc = cycling_data['tmp_count'].groupby([cycling_data['Team']]).sum()
cycling_data = cycling_data.drop(['tmp_count'], axis = 1)
cycling_data = pd.merge(cycling_data, ccc, on='Team', how='left')
#cycling_data = cycling_data[cycling_data['tmp_count'] > 5]
cycling_data['Team_dos'] = np.where(cycling_data['tmp_count'] < 8, 'Other', cycling_data['Team'])
cycling_data['Team_dos'] = np.where(cycling_data['Team_dos'] == 0, 'Other', cycling_data['Team_dos'])
cycling_data['Team'] = cycling_data['Team_dos']

# Drop country variable
cycling_data = cycling_data.drop(['Country', 'crawl_date', 'tmp_count', 'Team_dos'], axis = 1)
cycling_data.head()

,Rider,Team,GC,TT,Sprint,Climber,Classic,Age,Giro,Tour,Vuelta,Total,san_remo_win,flanders_win,lbl_win,lombardia_win,roubaix_win,strade_win,suisse_win,fleche_win,gent_win,paris_nice_win,e3_binckbank_win,amstel_win,dauphine_win,romandie_win
0,Primož Roglič,Team Jumbo-Visma,3581.0,2491.0,214.0,2572.0,512.0,30,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
1,Julian Alaphilippe,Deceuninck - Quick Step,1808.0,995.0,1336.0,3364.0,3479.0,27,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Alejandro Valverde,Movistar Team,12144.0,4575.0,5532.0,13660.0,11753.0,39,0.0,0.0,1.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,2.0,0.0
3,Jakob Fuglsang,Astana Pro Team,5595.0,1491.0,953.0,4845.0,2674.0,34,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
4,Egan Bernal,Team INEOS,2407.0,406.0,160.0,1889.0,677.0,23,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [0]:
#cycling_data.describe()
# Okay, let's make a column for winners of Paris-Nice type races
# Currently I'm thinking Paris - Nice, Tour de Suisse and Dauphine (Tierrento is not in the data set currently)
cycling_data['Paris_Nice_etc'] = cycling_data['paris_nice_win'] + cycling_data['dauphine_win'] + cycling_data['suisse_win'] + cycling_data['romandie_win']
cycling_data['Paris_Nice_etc_dos'] = np.where(cycling_data['Paris_Nice_etc'] > 0, 1, 0)
cycling_data['Paris_Nice_etc'] = cycling_data['Paris_Nice_etc_dos']
cycling_data = cycling_data.drop(['Paris_Nice_etc_dos'], axis = 1)

# Fix the age column type
cycling_data['Age'] = pd.to_numeric(cycling_data['Age'])

# Will need to drop those 3 columns now
cycling_data = cycling_data.drop(['paris_nice_win', 'dauphine_win', 'suisse_win', 'romandie_win'], axis = 1)

# Split the data sets
import numpy as np
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(cycling_data, test_size = 0.3, 
                                       random_state = 42)

# first let's make a fresh copy of our data and drop what we are trying
# to predict so it doesn't have any transmations applied to it
psg = train_set.drop('Paris_Nice_etc', axis=1)
psg_labels = train_set['Paris_Nice_etc'].copy()
psg_rider = train_set['Rider'].copy()


In [0]:
# Also from KMeans code
# This would be the best approach to removing special characters
#dictionary = {'ž':'z', 'č':'c', 'ü':'u', 'á':'a', 'é':'e', 'ö':'o', 'ó':'o', 'ç':'c',
#              'ń':'ń', 'í':'i', 'è':'e'}
#cycling_data.replace(dictionary, regex=True, inplace=True)

#d = psg.drop(['Rider'], axis=1) #, 'Team'

# let's normalize the data
psg['GC'] = preprocessing.scale(psg['GC'])
psg['Sprint'] = preprocessing.scale(psg['Sprint'])
psg['TT'] = preprocessing.scale(psg['TT'])
psg['Climber'] = preprocessing.scale(psg['Climber'])
psg['Classic'] = preprocessing.scale(psg['Classic'])
psg['Age'] = preprocessing.scale(psg['Age'])
psg['Giro'] = preprocessing.scale(psg['Giro'])
psg['Vuelta'] = preprocessing.scale(psg['Vuelta'])
psg['Tour'] = preprocessing.scale(psg['Tour'])
psg['Total'] = preprocessing.scale(psg['Total'])
psg['san_remo_win'] = preprocessing.scale(psg['san_remo_win'])
psg['lombardia_win'] = preprocessing.scale(psg['lombardia_win'])
psg['flanders_win'] = preprocessing.scale(psg['flanders_win'])
psg['lbl_win'] = preprocessing.scale(psg['lbl_win'])
psg['roubaix_win'] = preprocessing.scale(psg['roubaix_win'])
psg['strade_win'] = preprocessing.scale(psg['strade_win'])
#psg['suisse_win'] = preprocessing.scale(psg['suisse_win'])
psg['fleche_win'] = preprocessing.scale(psg['fleche_win'])
psg['gent_win'] = preprocessing.scale(psg['gent_win'])
#psg['Paris_Nice_etc'] = preprocessing.scale(psg['Paris_Nice_etc'])
psg['e3_binckbank_win'] = preprocessing.scale(psg['e3_binckbank_win'])
psg['amstel_win'] = preprocessing.scale(psg['amstel_win'])
#psg['dauphine_win'] = preprocessing.scale(psg['dauphine_win'])
#psg['romandie_win'] = preprocessing.scale(psg['romandie_win'])

# Algorithims prefer working with numbers generally, so let's convert the team
# proximity columns to numeric values
psg['Team'] = psg['Team'].astype(str)
housing_cat = psg[['Team']]
# Problem is if we use numeric examples for these variables algorithims
# will think they are near eachother. Instead we should make some one-hot
# encodings otherwise known as dummy variables for each category as it's own
# column
from sklearn.preprocessing import OrdinalEncoder
#ordinal_encoder = OrdinalEncoder()
from sklearn.preprocessing import OneHotEncoder
#cat_encoder = OrdinalEncoder()
#psg_cat_1hot = cat_encoder.fit_transform(housing_cat)

from sklearn.compose import ColumnTransformer
#cat_attribs = ['Team']

#full_pipeline = ColumnTransformer([
#      ('cat', OneHotEncoder(), cat_attribs),
#])

#psg_prepared = full_pipeline.fit_transform(psg) # applying this to pandas dataframe called housing

# not a fan of the approach above, changing it in the section below


In [0]:
# use pd.concat to join the new columns with your original dataframe
psg = pd.concat([psg,pd.get_dummies(psg['Team'], prefix='Team')],axis=1)

# now drop the original 'country' column (you don't need it anymore)
psg.drop(['Team'],axis=1, inplace=True)

In [143]:
# Drop rider name
psg = psg.drop(['Rider'], axis=1)

# rename so rest of coding works
psg_prepared = psg

# Load this for RMSE calcs
from sklearn.metrics import mean_squared_error

# Linear regression
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(psg_prepared, psg_labels)

# Let's test more powerful model
from sklearn.tree import DecisionTreeRegressor
# Build decision tree model
tree_reg = DecisionTreeRegressor()
tree_reg.fit(psg_prepared, psg_labels)

# we use 10 folds, which splits the data into 10 random folds
# and evaluates the decision tree model 10 times
# we test against a negative score cross-validation features expect
# a utility function (greater is better) rather than a cost function
# (where lower is better), this is the opposite of MSE
from sklearn.model_selection import cross_val_score
scores = cross_val_score(tree_reg, psg_prepared, psg_labels,
                         scoring='neg_mean_squared_error', cv=10)
tree_rmse_scores = np.sqrt(-scores)

# LEt's look at the results
def display_scores(scores):
  print('Scores:', scores)
  print('Mean:', scores.mean())
  print('Standard Deviation:', scores.std())

display_scores(tree_rmse_scores)  


Scores: [0.13363062 0.26726124 0.18898224 0.29880715 0.18898224 0.23145502
 0.13363062 0.23354968 0.19069252 0.19069252]
Mean: 0.2057683853057506
Standard Deviation: 0.05029414954837219


In [144]:
# let's compute the scores for the linear regression to be sure
lin_scores = cross_val_score(lin_reg, psg_prepared, psg_labels,
                             scoring = 'neg_mean_squared_error', cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)
# Woah, this is terrible after adding in the team dummy variables
# will need unique data set if using LR

Scores: [9.49295587e-02 4.09583336e+10 1.43773343e-01 1.97159048e-01
 1.08786423e-01 1.32541847e-01 5.96336514e-02 2.07231486e-01
 1.78994076e-01 1.23029368e-01]
Mean: 4095833362.4344625
Standard Deviation: 12287500086.888025


In [145]:
# Let's try a random forest model
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()
forest_reg.fit(psg_prepared, psg_labels)
psg_predictions = forest_reg.predict(psg_prepared)
forest_mse = mean_squared_error(psg_labels, psg_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse


0.05339295739572215

In [146]:
scores = cross_val_score(forest_reg, psg_prepared, psg_labels,
                         scoring='neg_mean_squared_error', cv=10)
forest_rmse_scores = np.sqrt(-scores)
display_scores(forest_rmse_scores)

Scores: [0.10028531 0.20466436 0.10854558 0.2246545  0.10625105 0.14651914
 0.03458943 0.18197402 0.1154123  0.12382539]
Mean: 0.13467210764541476
Standard Deviation: 0.053433610163165106


In [147]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
        'n_estimators': randint(low=50, high=120),
        'max_features': randint(low=25, high=52),
    }

forest_reg = RandomForestRegressor(random_state=42)
rnd_search = RandomizedSearchCV(forest_reg, param_distributions=param_distribs,
                                n_iter=30, cv=5, scoring='neg_mean_squared_error', random_state=42)
rnd_search.fit(psg_prepared, psg_labels)
# grid search can take a while, this is a randomized approach

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=100,
                              

In [148]:
cvres = rnd_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

0.14107610863767203 {'max_features': 31, 'n_estimators': 101}
0.140952798770822 {'max_features': 39, 'n_estimators': 110}
0.1406906557831469 {'max_features': 45, 'n_estimators': 73}
0.14235625927277007 {'max_features': 27, 'n_estimators': 71}
0.14004984908291432 {'max_features': 45, 'n_estimators': 51}
0.14272754360764217 {'max_features': 48, 'n_estimators': 79}
0.14367727455820598 {'max_features': 30, 'n_estimators': 51}
0.14090612386144366 {'max_features': 45, 'n_estimators': 82}
0.14356311848088474 {'max_features': 36, 'n_estimators': 107}
0.1443797117000251 {'max_features': 46, 'n_estimators': 98}
0.14368187145025021 {'max_features': 51, 'n_estimators': 108}
0.14014976524032569 {'max_features': 34, 'n_estimators': 109}
0.14278438291167864 {'max_features': 40, 'n_estimators': 64}
0.14114689207199427 {'max_features': 39, 'n_estimators': 111}
0.14360808084748974 {'max_features': 43, 'n_estimators': 104}
0.14265651213817943 {'max_features': 44, 'n_estimators': 113}
0.14064413633728762 

In [149]:
# LEt's go back to the default
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()
forest_reg.fit(psg_prepared, psg_labels)
psg_predictions = forest_reg.predict(psg_prepared)
forest_mse = mean_squared_error(psg_labels, psg_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

0.05575336815871569

In [154]:
# let's look at the relative importance of each attribute
feature_importances = forest_reg.feature_importances_
feature_importances
attributes = list(psg_prepared)
sorted(zip(feature_importances, attributes), reverse=True)

# Top riders
# let's look at top riders
sorted(zip(psg_predictions, psg_rider), reverse=True)

[(0.89, 'Richie Porte'),
 (0.84, 'Geraint Thomas'),
 (0.81, 'Tony Martin'),
 (0.79, 'Primož Roglič'),
 (0.78, 'Rui Costa'),
 (0.78, 'Roman Kreuziger'),
 (0.73, 'Carlos Betancur'),
 (0.72, 'Simon Špilak'),
 (0.71, 'Jakob Fuglsang'),
 (0.69, 'Sergio Henao'),
 (0.66, 'Egan Bernal'),
 (0.64, 'Marc Soler'),
 (0.61, 'Miguel Ángel López'),
 (0.57, 'Nairo Quintana'),
 (0.32, 'Dario Cataldo'),
 (0.29, 'Óscar Sevilla'),
 (0.26, 'Enric Mas'),
 (0.25, 'Wilco Kelderman'),
 (0.21, 'Vincenzo Nibali'),
 (0.18, 'Domenico Pozzovivo'),
 (0.17, 'Rafał Majka'),
 (0.17, 'Fabio Aru'),
 (0.16, 'Bauke Mollema'),
 (0.12, 'Thibaut Pinot'),
 (0.12, 'Mikel Nieve'),
 (0.11, 'Robert Gesink'),
 (0.11, 'Rigoberto Urán'),
 (0.11, 'Richard Carapaz'),
 (0.09, 'Simon Yates'),
 (0.09, 'Philippe Gilbert'),
 (0.09, 'Nelson Oliveira'),
 (0.08, 'Michał Kwiatkowski'),
 (0.07, 'Romain Bardet'),
 (0.07, 'Eduard Prades'),
 (0.06, 'Pierre Rolland'),
 (0.06, 'Nicolas Roche'),
 (0.06, 'Greg Van Avermaet'),
 (0.04, 'Tom-Jelte Slagter'

In [0]:
# let's evaluate the model on the test set
final_model = forest_reg
x_test = test_set.drop('Paris_Nice_etc', axis=1)
y_test = test_set['Paris_Nice_etc'].copy()
test_rider = test_set['Rider'].copy()

########
# Adjust the data
# let's normalize the data
x_test['GC'] = preprocessing.scale(x_test['GC'])
x_test['Sprint'] = preprocessing.scale(x_test['Sprint'])
x_test['TT'] = preprocessing.scale(x_test['TT'])
x_test['Climber'] = preprocessing.scale(x_test['Climber'])
x_test['Classic'] = preprocessing.scale(x_test['Classic'])
x_test['Age'] = preprocessing.scale(x_test['Age'])
x_test['Giro'] = preprocessing.scale(x_test['Giro'])
x_test['Vuelta'] = preprocessing.scale(x_test['Vuelta'])
x_test['Tour'] = preprocessing.scale(x_test['Tour'])
x_test['Total'] = preprocessing.scale(x_test['Total'])
x_test['san_remo_win'] = preprocessing.scale(x_test['san_remo_win'])
x_test['lombardia_win'] = preprocessing.scale(x_test['lombardia_win'])
x_test['flanders_win'] = preprocessing.scale(x_test['flanders_win'])
x_test['lbl_win'] = preprocessing.scale(x_test['lbl_win'])
x_test['roubaix_win'] = preprocessing.scale(x_test['roubaix_win'])
x_test['strade_win'] = preprocessing.scale(x_test['strade_win'])
#x_test['suisse_win'] = preprocessing.scale(x_test['suisse_win'])
x_test['fleche_win'] = preprocessing.scale(x_test['fleche_win'])
x_test['gent_win'] = preprocessing.scale(x_test['gent_win'])
#x_test['Paris_Nice_etc'] = preprocessing.scale(x_test['Paris_Nice_etc'])
x_test['e3_binckbank_win'] = preprocessing.scale(x_test['e3_binckbank_win'])
x_test['amstel_win'] = preprocessing.scale(x_test['amstel_win'])
#x_test['dauphine_win'] = preprocessing.scale(x_test['dauphine_win'])
#x_test['romandie_win'] = preprocessing.scale(x_test['romandie_win'])

# Algorithims prefer working with numbers generally, so let's convert the team
# proximity columns to numeric values
x_test['Team'] = x_test['Team'].astype(str)

# use pd.concat to join the new columns with your original dataframe
x_test = pd.concat([x_test,pd.get_dummies(x_test['Team'], prefix='Team')],axis=1)

# now drop the original 'country' column (you don't need it anymore)
x_test.drop(['Team'],axis=1, inplace=True)

# Drop rider name
x_test = x_test.drop(['Rider'], axis=1)


In [152]:
########

x_test_prepared = x_test
final_predictions = final_model.predict(x_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
final_rmse

0.12964374261799141

In [153]:
# let's look at top riders
sorted(zip(final_predictions, test_rider, y_test), reverse=True)

[(0.78, 'Tejay van Garderen', 0),
 (0.68, 'Chris Froome', 1),
 (0.64, 'Maxime Monfort', 0),
 (0.63, 'José Joaquín Rojas', 0),
 (0.6, 'Steven Kruijswijk', 0),
 (0.55, 'Ion Izagirre', 0),
 (0.47, 'Luis León Sánchez', 1),
 (0.35, 'Alejandro Valverde', 1),
 (0.26, 'Jürgen Roelandts', 0),
 (0.26, 'Francisco Mancebo', 0),
 (0.21, 'Diego Ulissi', 0),
 (0.15, 'Tom Dumoulin', 0),
 (0.12, 'Wout Poels', 0),
 (0.1, 'Edvald Boasson Hagen', 0),
 (0.09, 'Imanol Erviti', 0),
 (0.09, 'Dan Martin', 0),
 (0.06, 'Mikel Landa', 0),
 (0.05, 'Ilnur Zakarin', 1),
 (0.03, 'Pierre Latour', 0),
 (0.02, 'Tiesj Benoot', 0),
 (0.02, 'David de la Cruz', 0),
 (0.01, 'Mikkel Bjerg', 0),
 (0.01, 'Lars Boom', 0),
 (0.01, 'Davide Villella', 0),
 (0.01, 'Davide Formolo', 0),
 (0.0, 'Élie Gesbert', 0),
 (0.0, 'Zhandos Bizhigitov', 0),
 (0.0, 'Yves Lampaert', 0),
 (0.0, 'Yoann Paillot', 0),
 (0.0, 'Xianjing Lyu', 0),
 (0.0, 'Wouter Wippert', 0),
 (0.0, 'Winner Anacona', 0),
 (0.0, 'Valentin Madouas', 0),
 (0.0, 'Vadim Prons